In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("moon-tmhma").project("notes-dwac1-tdfg0")
version = project.version(1)
dataset = version.download("folder")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 43.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Notes-1 in folder:: 100%|██████████| 8398/8398 [00:02<00:00, 3756.67it/s]


In [3]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory


In [7]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = image_dataset_from_directory(
    "Notes-1/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = image_dataset_from_directory(
    "Notes-1/valid",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)

print(class_names)


Found 7585 files belonging to 11 classes.
Found 520 files belonging to 11 classes.
['1 Coin Rupee', '10', '10 Coin Rupee', '100', '2 Coin Rupee', '20', '200', '2000', '5 Coin Rupee', '50', '500']


In [8]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.fit(train_ds, validation_data=val_ds, epochs=35)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/35
238/238 ━━━━━━━━━━━━━━━━━━━━ 57s 172ms/step - accuracy: 0.2879 - loss: 1.8968 - val_accuracy: 0.4365 - val_loss: 1.3456
Epoch 2/35
238/238 ━━━━━━━━━━━━━━━━━━━━ 45s 71ms/step - accuracy: 0.5081 - loss: 1.2408 - val_accuracy: 0.4692 - val_loss: 1.1836
Epoch 3/35
238/238 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - accuracy: 0.5574 - loss: 1.1072 - val_accuracy: 0.5288 - val_loss: 1.1243
Epoch 4/35
238/238 ━━━━━━━━━━━━━━━━━━━━ 17s 70ms/step - accuracy: 0.5953 - loss: 1.0240 - val_accuracy: 0.5923 - val_loss: 1.0082
Epoch 5/35
238/238 ━━━━━━━━━━━━━━━━━━━━ 17s 71ms/step - accuracy: 0.6205 - loss: 0.9590 - val_accuracy: 0.5673 - val_loss: 1.0007
Epoch 6/35
238/238 ━━━━━━━━━━━━━━━━━━━━ 20s 70ms/step - accuracy: 0.6426 - loss: 0.9103 - val_accuracy: 0.5923 - val_loss: 0.9575
Epoch 7/35
238/238 ━━━━━━━━━━━━━━━━━━━━ 17s 71ms/step - accuracy: 0.6530 - loss: 0.8739 - val_accuracy: 0.6173 - val_loss: 0.9262
Epoch 8/35
238/238 ━━━━━━━━━━━━━━━━━━━━ 

In [10]:
model.save("currency_model.h5")